In [ ]:
!pip install fastai==0.7.0 --no-deps
!pip install scikit-learn==0.21.3
# fastai depends also on an older version of torch
!pip install torch==0.4.1 torchvision==0.1.9
!pip install torchtext==0.2.3
!pip show fastai

In [ ]:
from fastai.imports import *
from fastai.torch_imports import *
from fastai.io import *

In [ ]:
path = 'data/mnist/'
import os
os.makedirs(path, exist_ok=True)

In [ ]:
URL='https://github.com/mnielsen/neural-networks-and-deep-learning/raw/master/data/' 
FILENAME='mnist.pkl.gz'
import gzip

    
def load_mnist(filename):
    print(filename)
    return pickle.load(gzip.open(filename), encoding='latin-1')

In [ ]:
get_data(URL+FILENAME, path+FILENAME)
((x, y), (x_valid, y_valid), _) = load_mnist(path+FILENAME)

In [ ]:
type(x), x.shape, type(y), y.shape

## Normalize

In [ ]:
mean = x.mean()
std = x.std()
x=(x-mean)/std
mean, std, x.mean(), x.std()

In [ ]:
x_valid = (x_valid - mean)/std
x_valid.mean(), x_valid.std()

### Helper methods

In [ ]:
def show(img, title=None):
    plt.imshow(img, cmap='gray');
    if title is not None: plt.title(title)

In [ ]:
def plots(ims, figsize=(12,6), rows=2, titles=None):
    f = plt.figure(figsize = figsize)
    cols = len(ims)//rows
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None: sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], cmap='gray')

In [ ]:
?plt.imshow

## Plots

In [ ]:
x_valid.shape

In [ ]:
x_imgs = np.reshape(x_valid, (-1,28,28)); x_imgs.shape

In [ ]:
show(x_imgs[0,10:15,10:15])

In [ ]:
plots(x_imgs[:8], titles=y_valid[:8])

# Neural Net for Logistic Regression in PyTorch

In [ ]:
from fastai.metrics import *


import torch.nn as nn

In [ ]:
from fastai.model import *
from fastai.dataset import *

so this error is due to a bad python version, in 3.7 async is now a keyword..., need 3.6 python version to run this old fastai

In [ ]:
net = nn.Sequential(
    nn.Linear(28*28, 10),
    nn.LogSoftmax()
).cuda()
#this is a two layer neural net
#.cuda tells to run on GPU, if didnt say that, would run on CPU

In [ ]:
mp = ImageClasifireData.from_arrays(path, (x, y), (x_valid, y_valid))

In [ ]:
import sys
print(sys.version)

In [ ]:
loss = nn.NLLLoss() #negative low likelyhood loss = cross entropy. Either binary or categorical
metrics=[accuracy]
opt=optim.Adam(net.parameters())

### Fitting the model

In [ ]:
fit(net, md, epochs=1, crit=loss, opt=opt, metrics=metrics) #pythorch uses word criterion instead of loss

In [ ]:
preds = predict(net, md.val_dl)
preds.shape

In [ ]:
preds.argmax(axis=1)[:5] #argmax > figures out on this axis (for 10 in this case), return the index of the max value

In [ ]:
preds = preds.argmax(1) #saving this

In [ ]:
np.mean(preds == y_valid)

In [ ]:
plots(x_imgs[:8], titles=preds[:8])

#we have just built a logistic regression, it is not a deep neural net yet

In [ ]:
def get_weights(dims): return nn.Parameter(torch.randn(dims)/dims[0]) # because there are many layers, so need to make sure that the mean inputs not gonna change, otherwise its gonna diverge infinitely or converge it the number is bigger (gradient explosion)
#google: kaiming he initialization

In [ ]:
class LogReg(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1_w = get_weights(28*28, 10) # Layer 1 weights
        self.l1_b = get_weights(10)   # Layer 1 bias
    def forward(self, x): #in pytorch it has special meaning, it will get called when the layer gets calculated, its gonna get passed data form previous layer
        x = x.view(x.size(0), -1) #view = reshape
        x = torch.matmul(x, self.l1_w) +self.l1_b # Linear layer
        x = torch.log(torch.exp(x)/(torch.exp(x).sum(dim=0))) #this is a softmax (because we want all probabiliteis of outcomes to sumup to 1), so it behaves so tthat it returns something that behaves as probabilities
        return x
#there is a lot of info on pytorch website on how to create tensors and modify stuff

In [ ]:
#we define a * x, which are weights, then
#+b is called bias

In [ ]:
net2 = LogReg().cuda()
opt=optim.Adam(net2.parameters())

In [ ]:
fit(net2, md, epochs=1, crit=loss, opt=opt, metrics=metrics)

In [ ]:
1:19:30